In [8]:
%load_ext autoreload
%autoreload 2
import utils

import config


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
enriched = config.Grants.load_enriched()

In [31]:
enriched.columns

Index(['funder', 'last_updated', 'local_id', 'start_year',
       'nhmrc_funding_amount', 'purl', 'source', 'type', 'title', 'end_year',
       'participant_list', 'key', 'grant_summary', 'arc_for_primary',
       'arc_scheme_name', 'arc_announcement_administering_organisation', 'url',
       'arc_grant_status', 'arc_funding_at_announcement', 'arc_for',
       'arc_rfcd', 'funding_currency', 'crossref_grant_award',
       'funding_amount'],
      dtype='object')

In [28]:
enriched.key.str.split("/").map(lambda x: x[0]).value_counts()

key
arc      32338
orcid    29781
nhmrc    28547
doi         49
Name: count, dtype: int64

In [16]:
# Test the improved cleaning function (should have fewer warnings)
%load_ext autoreload
%autoreload 2

from prepare import GrantsCleaner
import config

print("=== TESTING IMPROVED CLEANING FUNCTION ===\n")

# Create cleaner and test it
cleaner = GrantsCleaner()
cleaned_df_v2 = cleaner.clean_enriched_data(config.Grants.enriched_path)

if cleaned_df_v2 is not None:
    print(f"\n=== FINAL SUMMARY ===")
    print(f"Total cleaned records: {len(cleaned_df_v2):,}")
    print(f"Memory usage: {cleaned_df_v2.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
    
    # Check sample records from each source
    print(f"\nSample records by source:")
    for source in cleaned_df_v2.source.unique():
        sample = cleaned_df_v2[cleaned_df_v2.source == source].iloc[0]
        print(f"  {source}: '{sample['title'][:60]}...' (Funder: {sample['funder_standardized']})")
else:
    print("Cleaning failed!")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
=== TESTING IMPROVED CLEANING FUNCTION ===

Loaded 90,715 total records from 4 sources
  - nhmrc.org: 28,547 records
  - arc.gov.au: 32,338 records
  - orcid.org: 29,781 records
  - crossref.org: 49 records
Processed NHMRC grants: 28,442 records
Processed ARC grants: 32,338 records
Processed ORCID grants: 12,365 records
Processed Crossref grants: 46 records
Loaded 90,715 total records from 4 sources
  - nhmrc.org: 28,547 records
  - arc.gov.au: 32,338 records
  - orcid.org: 29,781 records
  - crossref.org: 49 records
Processed NHMRC grants: 28,442 records
Processed ARC grants: 32,338 records
Processed ORCID grants: 12,365 records
Processed Crossref grants: 46 records


/Users/luhancheng/Desktop/research-link-technology-landscaping/modeling/prepare.py:509: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<NA>' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  orcid_grants.loc[:, 'funding_amount'] = None
/Users/luhancheng/Desktop/research-link-technology-landscaping/modeling/prepare.py:442: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_cleaned = pd.concat(cleaned_parts, axis=0, ignore_index=False)


Total cleaned records: 73,191

=== FINAL SUMMARY ===
Total cleaned records: 73,191
Memory usage: 60.8 MB

Sample records by source:
  nhmrc.org: 'Investigation of haematopoietic and leukemia stem cell self-...' (Funder: NHMRC)
  arc.gov.au: 'Australia's variable rainfall - how dry or wet can it really...' (Funder: ARC)
  orcid.org: 'The legal regulation of behaviour as a disability...' (Funder: ARC)
  crossref.org: 'TurExperience - Tourist experiences' impacts on the destinat...' (Funder: Fundação para a Ciência e a Tecnologia)


## Summary of Data Cleaning Improvements

### ✅ Unified Data Cleaning Function

The new `GrantsCleaner` class now handles all 4 data sources properly:

#### **Source Processing:**
1. **nhmrc.org** (28,442 records): NHMRC grants with funding amounts
2. **arc.gov.au** (32,338 records): ARC grants with funding amounts and FOR codes  
3. **orcid.org** (12,365 records): Additional Australian grants (ARC, NHMRC, Cancer Australia)
4. **crossref.org** (46 records): International grants with funding amounts

#### **Key Improvements:**
- **Merged two separate cleaning functions** into one unified approach
- **Handles all 4 sources** with appropriate logic for each
- **Standardized funding amount fields** from different source-specific columns
- **Added funder_standardized field** for consistent funder names
- **Improved filtering logic** to include relevant grants from all sources
- **Better data type handling** to reduce pandas warnings

#### **Output:**
- **Total cleaned records**: 73,191 (from 90,715 original)
- **Memory usage**: ~60.8 MB
- **Standardized columns**: title, grant_summary, funding_amount, start_year, end_year, funder, funder_standardized, for_primary, for, source

In [11]:
# Let's examine the structure by source
print("Data shape:", enriched.shape)
print("\nColumns:", list(enriched.columns))
print("\nSource counts:")
print(enriched.source.value_counts())

Data shape: (90715, 24)

Columns: ['funder', 'last_updated', 'local_id', 'start_year', 'nhmrc_funding_amount', 'purl', 'source', 'type', 'title', 'end_year', 'participant_list', 'key', 'grant_summary', 'arc_for_primary', 'arc_scheme_name', 'arc_announcement_administering_organisation', 'url', 'arc_grant_status', 'arc_funding_at_announcement', 'arc_for', 'arc_rfcd', 'funding_currency', 'crossref_grant_award', 'funding_amount']

Source counts:
source
arc.gov.au      32338
orcid.org       29781
nhmrc.org       28547
crossref.org       49
Name: count, dtype: int64


In [13]:
import pandas as pd

# Let's look at sample records from each source to understand the structure
for source in enriched.source.unique():
    print(f"\n=== {source} ===")
    sample = enriched[enriched.source == source].iloc[0]
    print(f"Sample title: {sample['title']}")
    print(f"Funder: {sample['funder']}")
    print(f"Grant summary exists: {pd.notna(sample['grant_summary']) and sample['grant_summary'] != ''}")
    
    # Check funding amount fields
    funding_fields = ['funding_amount', 'nhmrc_funding_amount', 'arc_funding_at_announcement', 'crossref_grant_award']
    for field in funding_fields:
        if field in sample and pd.notna(sample[field]):
            print(f"{field}: {sample[field]}")
    
    # Check FOR codes fields
    for_fields = ['arc_for_primary', 'arc_for']
    for field in for_fields:
        if field in sample and pd.notna(sample[field]):
            print(f"{field}: {sample[field]}")
    print("-" * 50)


=== nhmrc.org ===
Sample title: Investigation of haematopoietic and leukemia stem cell self-renewal.
Funder: National Health and Medical Research Council
Grant summary exists: True
nhmrc_funding_amount: 415218.3
--------------------------------------------------

=== arc.gov.au ===
Sample title: Australia's variable rainfall - how dry or wet can it really get?
Funder: arc.gov.au
Grant summary exists: True
arc_funding_at_announcement: 353000.0
arc_for_primary: 406.0
arc_for: 040104,0406,040605,040608
--------------------------------------------------

=== orcid.org ===
Sample title: The legal regulation of behaviour as a disability
Funder: Australian Research Council
Grant summary exists: False
--------------------------------------------------

=== crossref.org ===
Sample title: TurExperience - Tourist experiences' impacts on the destination image: searching for new opportunities to the Algarve.
Funder: Fundação para a Ciência e a Tecnologia
Grant summary exists: False
funding_amount:

In [14]:
# Let's analyze the key patterns for each source
print("=== KEY PATTERNS BY SOURCE ===\n")

for source in enriched.source.unique():
    subset = enriched[enriched.source == source]
    print(f"{source} ({len(subset):,} records):")
    
    # Funding amount patterns
    if source == 'nhmrc.org':
        funding_field = 'nhmrc_funding_amount'
    elif source == 'arc.gov.au':
        funding_field = 'arc_funding_at_announcement'
    elif source == 'crossref.org':
        funding_field = 'funding_amount'
    else:
        funding_field = None
    
    if funding_field:
        has_funding = subset[funding_field].notna().sum()
        print(f"  - Has {funding_field}: {has_funding:,} ({has_funding/len(subset)*100:.1f}%)")
    
    # Grant summary patterns
    has_summary = (subset['grant_summary'].notna() & (subset['grant_summary'] != '')).sum()
    print(f"  - Has grant summary: {has_summary:,} ({has_summary/len(subset)*100:.1f}%)")
    
    # FOR codes (only for ARC)
    if source == 'arc.gov.au':
        has_for_primary = subset['arc_for_primary'].notna().sum()
        has_for = subset['arc_for'].notna().sum()
        print(f"  - Has FOR primary: {has_for_primary:,} ({has_for_primary/len(subset)*100:.1f}%)")
        print(f"  - Has FOR codes: {has_for:,} ({has_for/len(subset)*100:.1f}%)")
    
    # Unique funders
    funders = subset['funder'].unique()
    print(f"  - Unique funders: {funders}")
    print()

=== KEY PATTERNS BY SOURCE ===

nhmrc.org (28,547 records):
  - Has nhmrc_funding_amount: 28,524 (99.9%)
  - Has grant summary: 12,148 (42.6%)
  - Unique funders: ['National Health and Medical Research Council'
 'Australian Research Council']

arc.gov.au (32,338 records):
  - Has arc_funding_at_announcement: 32,338 (100.0%)
  - Has grant summary: 21,505 (66.5%)
  - Has FOR primary: 32,263 (99.8%)
  - Has FOR codes: 32,263 (99.8%)
  - Unique funders: ['arc.gov.au']

orcid.org (29,781 records):
  - Has grant summary: 0 (0.0%)
  - Unique funders: ['Australian Research Council' 'Cancer Australia'
 'National Health and Medical Research Council' ...
 'Universitat Rovira i Virgili' 'National Academy of Sciences of Ukraine'
 'University of Technology Sydney - Health Future Development Grant (HFDG)']

crossref.org (49 records):
  - Has funding_amount: 46 (93.9%)
  - Has grant summary: 0 (0.0%)
  - Unique funders: ['Fundação para a Ciência e a Tecnologia' 'American Heart Association'
 'Wellcome 